In [1]:
# Grayscale/color Canny edge detection
# author: Muhammet Bastan, mubastan@gmail.com
# date: March 2012

import numpy, scipy, scipy.ndimage
from scipy.ndimage import filters
from scipy.ndimage import measurements
from numpy import *
from gradient import *

def getFractile(nms, th=1.0, fraction=0.30, bins=255):
    nzrind = nms >= th
    nzvals = nms[nzrind]
    minVal = nzvals.min()
    maxVal = nzvals.max()
    
    #figure(); hist(nzvals, bins=bins); draw()    
    H, e = numpy.histogram(nzvals, bins)
    
    nzr_frac = fraction*len(nzvals)    
    sum = 0.0
    i = 0
    while i < bins and sum < nzr_frac:
        sum += H[i]
        i += 1
    return (maxVal-minVal)*i/float(bins) + minVal


## nonmaximum suppression
# Gm: gradient magnitudes
# Gd: gradient directions, -pi/2 to +pi/2
# return: nms, gradient magnitude if local max, 0 otherwise
def nonmaxsupress(Gm, Gd, th=1.0):
    nms = zeros(Gm.shape, Gm.dtype)   
    h,w = Gm.shape    
    for x in range(1, w-1):
        for y in range(1, h-1):            
            mag = Gm[y,x]
            if mag < th: continue        
            teta = Gd[y,x]            
            dx, dy = 0, -1      # abs(orient) >= 1.1781, teta < -67.5 degrees and teta > 67.5 degrees
            if abs(teta) <= 0.3927: dx, dy = 1, 0       # -22.5 <= teta <= 22.5
            elif teta < 1.1781 and teta > 0.3927: dx, dy = 1, 1     # 22.5 < teta < 67.5 degrees
            elif teta > -1.1781 and teta < -0.3927: dx, dy = 1, -1  # -67.5 < teta < -22.5 degrees            
            if mag > Gm[y+dy,x+dx] and mag > Gm[y-dy,x-dx]: nms[y,x] = mag    
    return nms

### hysteresis thresholding
def hysteresisThreshold(nms, thLow, thHigh, binaryEdge = True):
    labels, n = measurements.label(nms > thLow, structure=ones((3,3)))
    for i in range(1, n):
        upper = amax(nms[labels==i])
        if upper < thHigh: labels[labels==i] = 0
    if binaryEdge: return 255*(labels>0)        
    else: return nms*(labels>0)

def detect_gray(image, thLow, thHigh, binaryEdge=True):
    Gm, Gd = gray_gradient(image)
    #print 'Gm max:', Gm.max(), mean(Gm)
    nms = nonmaxsupress(Gm, Gd, th=1.0)
    #fr = getFractile(nms, th=1.0, fraction=0.30, bins=255)
    #print 'Fractile:', fr, thLow, thHigh
    edge = hysteresisThreshold(nms, thLow, thHigh, binaryEdge)    
    return edge, nms

def detect_rgb(image, thLow, thHigh, gtype=0, binaryEdge=True):
    Gm, Gd = rgb_gradient(image, gtype) 
    #print 'Gm max:', Gm.max(), mean(Gm)
    nms = nonmaxsupress(Gm, Gd, th=1.0)
    #fr = getFractile(nms, th=1.0, fraction=0.50, bins=255)
    #print 'Fractile:', fr, thLow, thHigh
    edge = hysteresisThreshold(nms, thLow, thHigh, binaryEdge)
    return edge, nms

def detect_multichannel(images, thLow, thHigh, gtype=0, binaryEdge=True):
    Gm, Gd = multi_gradient(images, gtype) 
    #print 'Gm max:', Gm.max(), mean(Gm)
    nms = nonmaxsupress(Gm, Gd, th=1.0)
    #fr = getFractile(nms, th=1.0, fraction=0.50, bins=255)
    #print 'Fractile:', fr, thLow, thHigh
    edge = hysteresisThreshold(nms, thLow, thHigh, binaryEdge)
    return edge, nms
  
import sys
from pylab import *
if __name__ == "__main__":
    
    argc = len(sys.argv)
    
    if argc < 2:
        print 'Usage: python canny.py imageFile'
        sys.exit()
    
    
    
    imageC = scipy.misc.imread(sys.argv[1])
    imshow(imageC); title('input image'); draw()
    if len(imageC.shape)==2: gray()
    
    #image = scipy.misc.imread(sys.argv[1], True)
    image = scipy.misc.imread(sys.argv[1], False)
    sigma = 2.0
    image = scipy.ndimage.filters.gaussian_filter(image, sigma)
    
    imgMax=image.max()
    if imgMax > 255:
        image *= (255.0/65535)
        print 'Image max value:', imgMax
    
    tlow=100
    thigh=200
    if argc==4:
        tlow=float(sys.argv[2])
        thigh=float(sys.argv[3])
    
    #edge = detect_gray(image, tlow, thigh)
    edge = detect_rgb(image, tlow, thigh, 1)
    #edge = detect_rgb(image, 28, 57, 1)
    
    figure(); imshow(edge); gray()
    show()

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (3741867887.py, line 91)

In [3]:
# Gradient computation for grayscale/color Canny edge detection
# author: Muhammet Bastan, mubastan@gmail.com
# date: March 2012

import numpy, scipy, scipy.ndimage
from scipy.ndimage import filters
from numpy import *

# Sobel or Prewit gradient
def gradient(image, type=0):
    if type==0:
        Ix = filters.sobel(image, axis=1)
        Iy = filters.sobel(image, axis=0)
        return Ix, Iy
    else:
        Ix = filters.prewitt(image, axis=1)
        Iy = filters.prewitt(image, axis=0)
    return Ix, Iy

# gradient magnitudes and directions of a grayscale image, Gd: [-pi/2, +pi/2]
def gray_gradient(image):
    Gx, Gy = gradient(image)
    Gm = sqrt(Gx**2+Gy**2)
    Gd = arctan2(Gy, Gx)    
    Gd[Gd > 0.5*numpy.pi] -= numpy.pi
    Gd[Gd < -0.5*numpy.pi] += numpy.pi
    return Gm, Gd

# using the same approach of color gradient mag-ori computation for a gray image
def gray_gradient_tensor(image, sigma=0.5):
        
    g = image.astype('float32')
    #g = scipy.ndimage.filters.gaussian_filter(g, sigma)
    gx, gy = gradient(g)
    
    cxx = gx*gx
    cyy = gy*gy 
    cxy = 2*(gx*gy)     ## 2*cxy
    
    ## todo: smooth the color tensor
    cxx = scipy.ndimage.filters.gaussian_filter(cxx, sigma)
    cyy = scipy.ndimage.filters.gaussian_filter(cyy, sigma)
    cxy = scipy.ndimage.filters.gaussian_filter(cxy, sigma)
    
    cxx_cyy = cxx-cyy
    eps = 0.0000001
    d = sqrt( cxx_cyy**2 + cxy**2 + eps)
    
    ## largest eigenvalue -- derivative energy in the most prominent direction
    lambda1 = cxx + cyy + d          ## 0.5*... ?
    
    Gm = sqrt(lambda1 + eps)
    Gd = 0.5*arctan2(cxy, cxx_cyy)
    
    return Gm, Gd

# color gradient from multiple single channel images
# imgs: contains a list of single channel images (pixel values must be in the same range)
# output: Gm, gradient magnitude in the most prominent gradient direction (largest eigenvalue of color tensor)
# Gd: gradient directions [-pi/2, +pi/2]
# Reference: Van De Weijer, Joost and Gevers, Theo and Smeulders, Arnold WM, 'Robust Photometric Invariant Features from the Color Tensor', TIP 2006.
# This Py code was adapted from the Matlab code of the paper
def multi_gradient(imgs, gtype=0, sigma=0.5):
    if gtype==1: return multi_gradient_max(imgs)

    N=len(imgs)    
    # compute gradients
    # first channel    
    gx, gy = gradient(imgs[0])
    # structure tensor (color tensor in RGB images)
    cxx = gx*gx; cyy = gy*gy; cxy = gx*gy
    # remaining channels    
    for i in range(1, N):        
        gx, gy = gradient(imgs[i])
        cxx += gx*gx; cyy += gy*gy; cxy += gx*gy        
    cxy *= 2    ## 2*cxy
    
    ## smooth the structure/color tensor
    cxx = scipy.ndimage.filters.gaussian_filter(cxx, sigma)
    cyy = scipy.ndimage.filters.gaussian_filter(cyy, sigma)
    cxy = scipy.ndimage.filters.gaussian_filter(cxy, sigma)
    
    cxx_cyy = cxx-cyy
    eps = 1e-9
    d = sqrt( cxx_cyy**2 + cxy**2 + eps)
        
    ## largest eigenvalue -- derivative energy in the most prominent direction
    lambda1 = cxx + cyy + d          ## 0.5*... ?
        
    Gm = sqrt(lambda1 + eps)
    Gd = 0.5*arctan2(cxy, cxx_cyy)
    
    return Gm, Gd

# maximum gradient for each pixel in all the channels
def multi_gradient_max(imgs):
    N=len(imgs)      
    Gm,Gd = gray_gradient(imgs[0])
    for i in range(1, N):
        Gm2, Gd2 = gray_gradient(imgs[i])        
        ind= Gm2>Gm
        Gm[ind] = Gm2[ind]       
        Gd[ind] = Gd2[ind]
    return Gm, Gd

# gradient magnitude from an RGB image
# type: 0 (color tensor), else (max)
def rgb_gradient(image, gtype=0, sigma=0.5):
    r = image[:,:,0].astype('float32')
    g = image[:,:,1].astype('float32')
    b = image[:,:,2].astype('float32')
    
    imgs=[]
    imgs.append(r)
    imgs.append(g)
    imgs.append(b)
    
    if gtype == 0:
        return multi_gradient(imgs, sigma)
    else:
        return multi_gradient_max(imgs)

In [ ]:
import numpy as np
import cv2
import math


if __name__ == '__main__': 
   input_image = cv2.imread(r"D:\6m_NIC\images\trees.jpg",1) #Reads the input image
   output_image = input_image.copy()
   #Thresholding function
def threshfunc(in_img):
    img_out = np.copy(in_img)
    new_img = np.copy(in_img)
    vert_edge = np.array([[1,0,1],[0,0,0],[-1,0,-1]]) #Verticle array representing a Prewitt filter
    horz_edge = np.array([[-1,0,1],[-1,0,1],[-1,0,1]]) #Verticle array representing a Prewitt filter
    #Loops through the range of the image and moves the filter over each pixel for convolution
    for i in range(new_img.shape[0]):
        for j in range(in_img.shape[1]):
            #Checks to see if the filter mask is within range of the image
            #Does nothing if it is out of range
            if i<1 or i>(in_img.shape[0]-2) or j<1 or j>(in_img.shape[1]-2):
                pass
            else:
                vert = 0
                horz=0
                #Loops through the range of the 2d filter
                for x in range(-1,2):
                    for y in range(-1,2):
                        #Convolves the vertical and horizontal filters with the image
                        horz += new_img[i+x,j+y]*horz_edge[x+1,y+1]
                        vert += new_img[i+x,j+y]*vert_edge[x+1,y+1]
                #Finds the magnitude of the vertical and horizontal
                img_out[i,j] = int(math.sqrt(math.pow(horz,2)+math.pow(vert,2)))
                #Thresholding the output image, sets it to max value if its greater than a certain intensity, in this case 80
                if img_out[i,j] > 80:
                    img_out[i,j] = 255
                else:
                    #Threshold sets limit to 0 or black intensity of less than a certain arbitrary value, this case it is 80
                    img_out[i,j] = 0
    return img_out

def Edge_Dectection_func(in_img):
    new_img = np.copy(in_img)
    img_grey = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY) # line turns the colored rgb image to grayscale
    r = np.copy(img_grey) #Each of the 3 colors will have the same size as
    g = np.copy(img_grey)
    b = np.copy(img_grey) 
    #The loop gathers the intensity in the RBG image associated with each color and places them into an array, with each color having a coordinate
    for i in range (in_img.shape[0]):
        for j in range (in_img.shape[1]):
            r[i,j] = new_img[i,j][0]
            g[i,j] = new_img[i,j][1]
            b[i,j] = new_img[i,j][2] 
    #Passes each color array through the thresholding method
    r = threshfunc(r)
    g = threshfunc(g)
    b = threshfunc(b) 
    #Places each color into the image with their respective coordinate
    for x in range (in_img.shape[0]):
        for y in range (in_img.shape[1]):
            new_img[x,y][0]=r[x,y]
            new_img[x,y][1]=g[x,y]
            new_img[x,y][2]=b[x,y] 

    return new_img

if __name__ == '__main__': 
   output_image  = Edge_Dectection_func(input_image) #calls upon edge detection function
   cv2.imwrite('threshed_snow.png',output_image)
   cv2.waitKey(0)
   cv2.destroyAllWindows()

C:\Users\DELL\AppData\Local\Temp\ipykernel_21772\1429928309.py:32: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 260 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  img_out[i,j] = int(math.sqrt(math.pow(horz,2)+math.pow(vert,2)))
C:\Users\DELL\AppData\Local\Temp\ipykernel_21772\1429928309.py:32: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 275 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  img_out[i,j] = int(math.sqrt(math.pow(horz,2)+math.pow(vert,2)))
C:\Users\DELL\AppData\Local\Temp\ipykernel_21772\1429928309.py:32: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays